<a href="https://colab.research.google.com/github/CameronKenworthyCode/NEWICC/blob/main/PLSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymongo[srv]
!pip install dnspython
!pip install requests
!pip install bs4
!pip install html2text
!pip install -U -q PyDrive

import html2text
from ast import literal_eval
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId
import numpy as np
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

auth.authenticate_user() #establish connection to a gdrive to write the eventual csv file to
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

drive.mount('/content/drive/', force_remount=True)

file_list = gdrive.ListFile(
    {'q': "'1DWFETkLt-dWcA7CfymejJe7O-UJxSR4B' in parents and trashed=False"}).GetList() 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 7.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive/


In [ ]:
file_list[0]

GoogleDriveFile({'kind': 'drive#file', 'id': '1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1', 'etag': '"MTY1ODAxNTI0NTI1MA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1', 'webContentLink': 'https://drive.google.com/uc?id=1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1&export=download', 'alternateLink': 'https://colab.research.google.com/drive/1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1', 'embedLink': 'https://drive.google.com/file/d/1KNxBQXIKc2rGgIcjR3wh95F0eh1fuxr1/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/vnd.google.colaboratory', 'title': 'PLSS.ipynb', 'mimeType': 'application/vnd.google.colaboratory', 'description': 'Colaboratory notebook', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'copyRequiresWriterPermission': False, 'createdDate': '2022-07-16T23:41:41.265Z', 'modifiedDate': '2022-07-16T23:47:25.250Z', 'modifiedByMeDate': '2022-07-16T23:47:25.250Z', 

# TODO

- generate URLS
- batch query https://gis.blm.gov/arcgis/rest/services/Cadastral/BLM_Natl_PLSS_CadNSDI/MapServer/exts/CadastralSpecialServices/GetLatLon?trs=
- save geometry long/lat coords
  - number of coords given varies per section
    - get corners
      - get all points along each edge
    - infer outline of township range from edges of sections
- infer shape from coords
- save all as csv
- code to calculate if coord is in shape

#Generate URL Queries

TOWN: 1N -> 41N

RANGE: 16W -> 47E


In [54]:
# generate set of strings in form WA330##0N0##0E0SN##0 or WA330##0N0##0W0SN##0
def generate_url(town, _range):
  t_ = town[:-1]
  if int(t_) < 10:
    t_ = '0' + t_
  r_ = _range[:-1]
  if int(r_) < 10:
    r_ = '0' + r_
  r_ = r_ + '0' + _range[-1:]
  base_string = 'WA330'+ t_ + '0N0' + r_ + '0SN'
  section_list = []
  for i in range(1, 60):
    s_ = str(i) + '0'
    if i < 10:
      s_ = '0' + str(i) + '0'
    section_list.append(base_string+s_)
  
  return section_list
  

In [ ]:
trs = []

for i in range(1,42):
  for j in range(1,17):
    trs.append([str(i)+'N', str(j)+'W'])

for i in range(1, 42):
  for j in range(1, 48):
    trs.append([str(i)+'N', str(j)+'E'])

url_dict = {}
for pair in trs:
  key = pair[0] + '-' + pair[1]
  value = generate_url(pair[0], pair[1])
  url_dict[key] = value

# go through each T/R pair, and take the next 25 section urls to build a query, save the results and continue to the next 25
PLSS_dict = {}
base_url = 'https://gis.blm.gov/arcgis/rest/services/Cadastral/BLM_Natl_PLSS_CadNSDI/MapServer/exts/CadastralSpecialServices/GetLatLon?trs='
built_url = ''
base_url_end = '&returnalllevels=&f=pjson'
count = 0
for i in range(len(trs)):
  for j in range(0, 59):
    if count == 25:
      print(base_url+built_url+base_url_end)
      #query_result = query(base_url+build_url+base_url_end) # send query
      #PLSS_dict[trs[i][0] + '-' + trs[i][1] + '-' + j] = query_result
      built_url = ''
      count = 0
    count += 1
    built_url = built_url + url_dict[trs[i][0] + '-' + trs[i][1]][j] + '+%7C+'

In [63]:
built_url = ''
built_url = built_url + url_dict[trs[0][0] + '-' + trs[0][1]][0] + '+%7C+'

In [ ]:
url_dict[trs[0][0] + '-' + trs[0][1]]

https://gis.blm.gov/arcgis/rest/services/Cadastral/BLM_Natl_PLSS_CadNSDI/MapServer/exts/CadastralSpecialServices/GetLatLon?trs=

WA33 0**TT**0N 0**RR**0E 0SN**SS**0

25 TRS per query

```
WA330270N0310E0SN060+%7C+ # 27/31/06
WA330160N0260E0SN120+%7C+
WA330170N0300E0SN170+%7C+
WA330170N0280E0SN300+%7C+
WA330160N0240E0SN130+%7C+
WA330160N0250E0SN080+%7C+
WA330160N0240E0SN130+%7C+
WA330200N0280E0SN020+%7C+
WA330210N0280E0SN340+%7C+
WA330190N0250E0SN040+%7C+
WA330200N0230E0SN040+%7C+
WA330200N0280E0SN020+%7C+
WA330210N0280E0SN340+%7C+
WA330170N0300E0SN080+%7C+
WA330180N0300E0SN240+%7C+
WA330180N0300E0SN260+%7C+
WA330200N0280E0SN020+%7C+
WA330200N0280E0SN020+%7C+
WA330190N0290E0SN280+%7C+
WA330190N0290E0SN280+%7C+
WA330190N0290E0SN330+%7C+
WA330170N0240E0SN100+%7C+
WA330170N0240E0SN110+%7C+
WA330190N0270E0SN080+%7C+
WA330200N0250E0SN110+%7C+
```



&returnalllevels=&f=pjson&__ncforminfo=7x7RLQJqpDQ92T4PQI_-wSk34fhSp0krCuCkP5nOMe9rVpPf2stN7FAL0CQsbN_c62SdT15Xs7Rs5U4Xfe3epxz0dE12hq0HyWAsrAz430mfnjihcVff8g%3D%3D

In [ ]:
#